# Dataset Tool

In [ ]:
import pandas as pd
csv_path = 'H://reddit/corgis/'
df = pd.read_csv(os.path.join(csv_path, 'submissions_clean.csv'))
img_dir = os.path.join(csv_path, 'img')
label_dir = os.path.join(csv_path, 'labels')

pathlib.Path(img_dir).mkdir(parents=True, exist_ok=True)
pathlib.Path(label_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
import numpy as np
import shutil
for item in df.iterrows():
    item = item[1]
    filename = item['id']
    if os.path.isfile(os.path.join(csv_path, 'images', filename+'.jpg')):
        filename += '.jpg'
    elif os.path.isfile(os.path.join(csv_path, 'images', filename+'.jpeg')):
        filename += '.jpeg'
    elif os.path.isfile(os.path.join(csv_path, 'images', filename+'.png')):
        filename += '.png'
    else:
        print(filename)
        
    bbox = np.array([[item['bboxX'], item['bboxY'], item['bboxX'] + item['bboxSize'], item['bboxY'] + item['bboxSize']]])
    labels = np.array([18])
    scores = np.array([1.0])
    dick = {'boxes': bbox, 'labels': labels, 'scores': scores}
    pickle.dump(dick, open(os.path.join(label_dir, filename.split('.')[0] + '.pkl'), 'wb'))
    shutil.copyfile(os.path.join(csv_path, 'images', filename), os.path.join(img_dir, filename))

In [ ]:
for picl in os.listdir(pred_dir):
    prediction = pickle.load(open(os.path.join(pred_dir, picl), 'rb'))
    for key, val in prediction.items():
        prediction[key] = val.detach().numpy()
    pickle.dump(prediction, open(os.path.join(pred_dir, picl), 'wb'))

## Setup Faster RCNN Model

In [2]:
import torchvision
import torch
import os
import numpy as np
from PIL import Image, ImageDraw
from torchvision.transforms import ToTensor
from tqdm import tqdm
import pathlib
import pickle
import hashlib

image_dir = './corgis'
pred_dir = os.path.join(image_dir, 'labels')
super_dir = os.path.join(image_dir, 'supervision')

In [ ]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# For training
images, boxes = torch.rand(4, 3, 600, 1200), torch.rand(4, 11, 4)

# For inference
model.eval()
predictions = model([images[0]])

## Predict bboxes

In [ ]:
pathlib.Path(pred_dir).mkdir(parents=True, exist_ok=True)
image_names = os.listdir(image_dir)

for image_name in tqdm(os.listdir(image_dir)):
    if os.path.isdir(os.path.join(image_name, file_name)): continue
    image = Image.open(os.path.join(image_dir, image_name))
    image_tensor = ToTensor()(image).unsqueeze(0)
    model.eval()
    predictions = model(image_tensor)[0]
    for key, value in predictions.items():
        predictions[key] = value.detach().numpy()
        
    pickle.dump(predictions, open(os.path.join(pred_dir, image_name.split('.')[0] + '.pkl'), "wb" ))

# Clean (videos, dups, small res)
Todo: can not open

In [ ]:
def remove_videos(image_dir):
    file_names = os.listdir(image_dir)
    for file_name in file_names:
        if os.path.isdir(os.path.join(image_dir, file_name)):
            continue
        ext = file_name.split('.')
        if len(ext) < 2 or ext[1] not in ['jpg', 'jpeg', 'png']:
            os.remove(os.path.join(image_dir, file_name))
            
remove_videos(image_dir)

In [ ]:
def create_size_dict(image_dir):
    image_names = os.listdir(image_dir)
    size_dict = {}
    for image_name in tqdm(image_names):
        if os.path.isdir(os.path.join(image_dir, image_name)): continue
        
        image = Image.open(os.path.join(image_dir, image_name))
        w,h = image.size
        size_str = f'{w}x{h}'
        if size_str not in size_dict.keys():
            size_dict[size_str] = [image_name]
        else:
            size_dict[size_str] = size_dict[size_str] + [image_name]
            
    return size_dict

size_dict = create_size_dict(image_dir)

In [ ]:
removed = 0
remove_keys = []
for size in size_dict.keys():
    w, h = [int(x) for x in size.split('x')]
    if w < 700 and h < 700:
        for image_name in size_dict[size]:
            os.remove(os.path.join(image_dir, image_name))
            removed += 1
        remove_keys.append(size)
        
for key in remove_keys:
    size_dict.pop(key, None)

print(removed)

In [ ]:
removed = 0
for size in size_dict.keys():
    if len(size_dict[size]) < 2:
        continue
    
    hashes = []
    image_files = []
    for image_file in size_dict[size]:
        hash_md5 = hashlib.md5()
        with open(os.path.join(image_dir, image_file), "rb") as f:
            for chunk in iter(lambda: f.read(4096), b""):
                hash_md5.update(chunk)
        hashes.append(hash_md5.hexdigest())
        image_files.append(image_file)
        
    unique_hashes = set(hashes)
    for h in unique_hashes:
        cnt = hashes.count(h)
        if cnt > 1:
            idxs = [i for i in range(len(hashes)) if hashes[i] == h]
            idxs.pop(0)
            for idx in idxs:
                os.remove(os.path.join(image_dir, image_files[idx]))
                removed +=1

print(removed)

## Filter bassed on bboxes

In [ ]:
from IPython.display import display

# https://stackoverflow.com/questions/25349178/calculating-percentage-of-bounding-box-overlap-for-image-detector-evaluation/42874377
def get_overlap(bb1, bb2):
    x_left = max(bb1[0], bb2[0])
    y_top = max(bb1[1], bb2[1])
    x_right = min(bb1[2], bb2[2])
    y_bottom = min(bb1[3], bb2[3])

    if x_right < x_left or y_bottom < y_top:
        return 0.0, 0.0

    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    bb1_area = (bb1[2] - bb1[0]) * (bb1[3] - bb1[1])
    bb2_area = (bb2[2] - bb2[0]) * (bb2[3] - bb2[1])

    overlap1 = intersection_area/bb1_area
    overlap2 = intersection_area/bb2_area
    return overlap1, overlap2

image_names = os.listdir(image_dir)
pathlib.Path(super_dir).mkdir(parents=True, exist_ok=True)

for image_name in tqdm(os.listdir(image_dir)):
    if os.path.isdir(os.path.join(image_dir, image_name)): continue
        
    basename, ext = image_name.split('.')
    
    prediction = pickle.load(open(os.path.join(pred_dir, basename+'.pkl'), 'rb'))
    print(prediction)
    break
    #print(prediction)

    #print(image_name)
    
    im = Image.open(os.path.join(image_dir, image_name))
    draw = ImageDraw.Draw(im)
    dog_idxs = np.argwhere((prediction['labels'] == 18) & (prediction['scores'] > 0.9)).flatten()
    dog_bboxes = prediction['boxes']
    
    dog_idxs_nms = []
    if True:
        overlap_thresh = 0.7
        area = lambda bbox: (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])
        for i, idx1 in enumerate(dog_idxs):
            delete = False
            for j, idx2 in enumerate(dog_idxs):
                if i == j: continue

                # Check IOU
                bbox1 = dog_bboxes[idx1]
                bbox2 = dog_bboxes[idx2]
                overlap1, overlap2 = get_overlap(bbox1, bbox2)

                # If overlap and current bbox is smaller than box compared to, then discard this bbox
                if overlap1 > overlap2 and overlap1 > overlap_thresh:
                    delete = True
                    break
            if not delete: dog_idxs_nms.append(idx1)
    else:
        dog_idxs_nms = dog_idxs
        
    if len(dog_idxs_nms) != 1:
        os.rename(os.path.join(image_dir, image_name), os.path.join(super_dir, image_name))
    
    #for d in dog_idxs_nms:
    #    draw.rectangle(prediction['boxes'].detach().numpy()[d])
        
    #display(im)
    
    #print(dog_mask)
    #i += 1
    #if i == 30: break
print(i)

# Crop

In [ ]:
image_names = os.listdir(image_dir)
crop_dir = os.path.join(image_dir, 'crop')
pathlib.Path(crop_dir).mkdir(parents=True, exist_ok=True)

k = 0
for image_name in tqdm(os.listdir(image_dir)):
    if os.path.isdir(os.path.join(image_dir, image_name)): continue
    basename, ext = image_name.split('.')
    
    prediction = pickle.load(open(os.path.join(pred_dir, basename+'.pkl'), 'rb'))
    im = Image.open(os.path.join(image_dir, image_name)).convert('RGB')
    dog_idxs = np.argwhere((prediction['labels'] == 18) & (prediction['scores'] > 0.7)).flatten()
    dog_bboxes = prediction['boxes']
    
    dog_idxs_nms = []
    if True:
        overlap_thresh = 0.7
        area = lambda bbox: (bbox[2] - bbox[0]) * (bbox[3] - bbox[1])
        for i, idx1 in enumerate(dog_idxs):
            delete = False
            for j, idx2 in enumerate(dog_idxs):
                if i == j: continue

                # Check IOU
                bbox1 = dog_bboxes[idx1]
                bbox2 = dog_bboxes[idx2]
                overlap1, overlap2 = get_overlap(bbox1, bbox2)

                # If overlap and current bbox is smaller than box compared to, then discard this bbox
                if overlap1 > overlap2 and overlap1 > overlap_thresh:
                    delete = True
                    break
            if not delete: dog_idxs_nms.append(idx1)
    else:
        dog_idxs_nms = dog_idxs
        
    idx = dog_idxs_nms[0]
    bbox = dog_bboxes[idx]
    w, h = im.size
    min_side_img = min(w, h)
    
    # Scale to at least 1024 pixels
    scale_f = 1.0
    if min_side_img < 1024:
        scale_f = 1024/min_side_img
        im = im.resize((int(w*scale_f), int(h*scale_f)))
        w, h = im.size
    
    bbox *= scale_f
    min_side_img = min(w, h)
    
    max_side_bbox = int(max(bbox[2] - bbox[0], bbox[3] - bbox[1]))

    # Ensure that crop is at least 1024, fits into image and, if possible, has size max_side_bbox + max_size_img/10
    crop_size = min(min_side_img, max(1024, max_side_bbox + min_side_img/10))

    bbox_center_x = bbox[0] + (bbox[2]-bbox[0])/2
    bbox_center_y = bbox[1] + (bbox[3]-bbox[1])/2
    crop_x = max(0, bbox_center_x-crop_size/2)
    crop_y = max(0, bbox_center_y-crop_size/2)
    crop_x_over = w - (crop_x + crop_size)
    crop_y_over = h - (crop_y + crop_size)
    if crop_x_over < 0:
        crop_x += crop_x_over
    if crop_y_over < 0:
        crop_y += crop_y_over
        
    #draw = ImageDraw.Draw(im)
    #draw.point([bbox_center_x, bbox_center_y], fill='red')
    #draw.rectangle(bbox, outline='blue')
    #draw.rectangle((crop_x, crop_y, crop_x + crop_size, crop_y + crop_size), outline='red')
    #display(im)
    #display(im)
    #display(im.crop(crop_x, crop_y, crop_x + crop_size, crop_y + crop_size))
    im_crop = im.crop((crop_x, crop_y, crop_x + crop_size, crop_y + crop_size))
    im_crop.save(os.path.join(crop_dir, image_name), "JPEG")
        

In [ ]:
from imagededup.methods import PHash
phasher = PHash()

# Generate encodings for all images in an image directory
encodings = phasher.encode_images(image_dir='corgis')

# Find duplicates using the generated encodings
duplicates = phasher.find_duplicates(encoding_map=encodings)

# plot duplicates obtained for a given file using the duplicates dictionary
from imagededup.utils import plot_duplicates
plot_duplicates(image_dir='corgis',
                duplicate_map=duplicates,
                filename='ukbench00120.jpg')

In [ ]:
plot_duplicates(image_dir='corgis',
                duplicate_map=duplicates,
                filename='100082095_112963816898945_1812614927344669390_n.jpg')

In [ ]:
x = input()
print(x)

In [ ]:
duplicate_keys = [key for key in duplicates.keys() if len(duplicates[key]) != 0]
print(len(duplicate_keys))
to_be_deleted = []
for key in duplicates.keys():
    if key in duplicate_keys:
        plot_duplicates(image_dir='corgis',
                duplicate_map=duplicates,
                filename=key)
        
        delete = input('Press x if duplicates should be deleted')
        if delete == 'x':
            to_be_deleted.extend(duplicates[key])
            for dup in duplicates[key]:
                duplicate_keys.remove(dup)

In [ ]:
duplicate_keys = [key for key in duplicates.keys() if len(duplicates[key]) != 0]
#removed = 0
for key in duplicates.keys():
    if key in duplicate_keys:
        for dup in duplicates[key]:
            try:
                os.remove(os.path.join(image_dir, dup))
                duplicate_keys.remove(dup)
            except:
                print('ohoh')
                pass
            removed += 1

In [ ]:
from tqdm import tqdm
import os
profiles = ['genthecorgi','corgi_shumai','corgi_foxy','corgi.mob','corgi_nola','currydecorgi','lecorgi','thesportycorgi','iggy_corgi','corgiofigs','pino.the.corgi','corgiexotic','sake_thecorgi','myfavcorgi','ralphthecorgi','topithecorgi','corgithings','the_corgi_yoshi','tofu_corgi','jamie_the_dutch_corgi','corgisonstairs','corgi.kevin','corgi.persey','mac_corgi','poncho.the.corgi','mabelthetricorgi','corgimowgli','lychee_the_corgi','corgisftw','bootsythecorgi']
#'lacorgi', 'madmax_fluffyroad','corgiarya','corgis.hub',
for profile in tqdm(profiles):
    os.system(f'instaloader --no-videos --no-video-thumbnails --no-captions --no-metadata-json --no-profile-pic {profile}')

 13%|██████████▋                                                                     | 4/30 [23:42<2:20:51, 325.07s/it]